In [1]:
!nvidia-smi

Mon Mar 25 15:48:36 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN Xp            Off  | 00000000:01:00.0 Off |                  N/A |
| 23%   27C    P8    16W / 250W |      0MiB / 12195MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from os import path as p
import os
from sklearn.metrics import mean_squared_error

In [3]:
data_dir = '/storage/yw18581/data/'
data_folder = os.path.join(data_dir, 'train_validation_test')

In [4]:
X_train = np.load(os.path.join(data_folder, 'Xy_train_dist.npz'))["y"]
y_train = np.load(os.path.join(data_folder, 'Xy_train_dist.npz'))["dist"]

In [5]:
X_val = np.load(os.path.join(data_folder,'Xy_val_dist.npz'))["y"]
y_val = np.load(os.path.join(data_folder, 'Xy_val_dist.npz'))["dist"]

In [6]:
X_test = np.load(os.path.join(data_folder, 'Xy_test_dist.npz'))["y"]
y_test = np.load(os.path.join(data_folder, 'Xy_test_dist.npz'))["dist"]

In [7]:
X_pred = np.load(os.path.join(data_dir, 'trained_models/UNet_training_generator_1500epochs/Xy_test_predicted_UNet.npz'))['y']
y_pred = np.load(os.path.join(data_dir, 'trained_models/UNet_training_generator_1500epochs/Xy_test_predicted_UNet.npz'))['dist']

In [8]:
def cut_X(arr, reshape = None):
    x_cut = arr[:,960:1300,600:]
    if reshape:
        if len(x_cut.shape)>3:
            x_cut = x_cut[...,0]
            x_cut_out = x_cut.reshape(x_cut.shape[0],x_cut.shape[1]*x_cut.shape[2])
    else:
        x_cut_out = x_cut
    return x_cut_out

In [9]:
def reshape_RF(arr):
    arr_RF = arr.reshape((arr.shape[0], arr.shape[1]*arr.shape[2]))
    return arr_RF

In [10]:
X_train_cut = cut_X(X_train)

In [11]:
X_val_cut = cut_X(X_val)

In [12]:
X_test_cut = cut_X(X_test)

In [13]:
X_pred_cut = cut_X(X_pred)

In [14]:
X_train_RF = reshape_RF(X_train_cut)
X_val_RF = reshape_RF(X_val_cut)

In [15]:
N_EPOCHS = 30
BATCH_SIZE = 8

CHECKPOINT_FOLDER_PATH = p.join(data_dir, 'trained_models')
TASK_NAME = 'Leaf_position_regression'
TASK_FOLDER_PATH = p.join(CHECKPOINT_FOLDER_PATH, TASK_NAME)

if not p.exists(TASK_FOLDER_PATH):
    os.makedirs(TASK_FOLDER_PATH)

## Random Forest Regressor

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [17]:
rf1 = RandomForestRegressor(max_depth=10, n_estimators=100, n_jobs=-1, 
                              criterion='mse'  # default
                             )
rf1.fit(X_train_RF, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [18]:
# Training Performance
rf1.score(X_train_RF, y_train)  # will calculate MSE by default

0.9999442126026229

In [19]:
preds = rf1.predict(X_test_cut.reshape(240, 340*800))

In [25]:
preds

array([25. , 10. ,  2. ,  4. ,  2. , 25. ,  3.8, 25. ,  4. , 10. , 10. ,
        2. ,  2. , 10. ,  2. ,  2. ,  4. , 10. , 10. ,  4. ,  4. , 10. ,
       10. ,  4. ,  2. ,  2. , 10. ,  4. ,  4. ,  4. ,  3.8, 10. , 10. ,
       10. ,  4. ,  4. ,  4. ,  2. ,  3.8, 10. , 25. ,  2. , 23.5,  4. ,
       25. ,  3.6,  4. ,  2. ,  2. ,  2. , 10. , 25. ,  4. ,  2. , 25. ,
        4. , 25. , 25. , 25. ,  4. ,  4. ,  2. ,  2. ,  2. ,  2. , 25. ,
        2. ,  4. ,  4. ,  3.6,  4. ,  2. , 25. ,  2. ,  2. ,  4. ,  2. ,
        2. , 25. , 10. , 25. ,  2. , 10. ,  2. ,  4. , 25. , 25. ,  2. ,
       25. , 10. ,  4. ,  3.8,  2. ,  4. , 25. , 10. ,  4. , 25. ,  2. ,
        2. ,  4. ,  2. , 10. ,  2. , 10. , 10. , 25. , 25. , 25. , 25. ,
        4. ,  9.4,  2. , 25. , 10. , 25. ,  2. ,  2. , 10. ,  2. ,  4. ,
       25. , 25. , 25. ,  4. , 25. ,  3.6,  2. , 10. , 10. ,  2. , 25. ,
        4. ,  2. , 25. , 10. ,  7.6,  4. ,  4. , 25. , 25. , 10. ,  4. ,
       10. , 10. ,  4. , 10. , 10. ,  4. ,  4. , 23

In [20]:
mean_squared_error(y_test, preds)

0.023111249999999986

In [21]:
from sklearn.externals import joblib

In [22]:
filename = os.path.join(TASK_FOLDER_PATH,'leaf_regression_RF_100est_10depth.pkl')
_ = joblib.dump(rf1, filename, compress=9)